# Buyers Journey Content Heat Map

**Disclaimer:** *This work is based on a project I did for Highspot. I use simulated data since I cannot share or distribute any company data or information*

Buyers undergo multiple stages before they make a purchase: they first become aware of a product; then they show interest in the product; next they get invloved is seeking information regarding the product; and finally, they purchase the product. One notable framework explaining this buyers journey is the AIDA model. Things get pretty complicated in the B2B world (where the customers are businesses), but the framework is pretty much the same. There, the marketing and sales teams use a slightly complex framework to target the potential customers more effectively. The framework they use is generally referred to as sales (or marketing, or B2B lead) funnel. The concept varies from one company to another; the framework, however, is pretty much similar and includes the following stages:

- Prospect
- MQL
- SQL
- Opportunity
- Closed Won/ Lost/ No Decision

As mentioned earlier, the model used in many companies is more complicated and has more stages; however, the analysis is the same regardless of the particular model or the number of stages.

The sales team uses the content (generally created by the marketing team) to target each portential customer and then monitor they engagement with the content. Using the content for such purposes is generally referred to as *pitching*. The Highspot platform makes is possible to track the engagement with the content using the four metrics of *opened*, *viewed*, *shared* and *downloaded*. The higher the ratio of each of the metrics to the number of pitches, the more potential customers are engaged with the content. Therefore, we decided to calculated the ratio for all of the content and use it as a measure of the engagement with that item.

In [1]:
# importing necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objs as go
import plotly.figure_factory as ff
import plotly.tools as tls
import plotly.io as pio
import os

from matplotlib.colors import ListedColormap
from plotly.offline import init_notebook_mode, iplot, plot
from IPython.display import Image, display


# settings
pd.set_option('display.max_columns', None)
plt.tight_layout()
sns.set_style("whitegrid")
init_notebook_mode(connected=True)

%matplotlib inline
%config InlineBackend.figure_format = 'retina'


# creating a directory for images
if not os.path.exists('images'):
    os.mkdir('images')

color = '#298DE1'  # Highspot color
scale = 2  # scale for saving images

In [2]:
# generate data
n = 100  # number of items

item_names = 'Item ' + \
    pd.Series(np.arange(n), name='item_name').astype(str)  # item names
item_names = pd.concat([item_names]*5, ignore_index=True)

stages = pd.Series(
    ['Prospect', 'MQL', 'SQL', 'Opportunity', 'Closed Won'],
    name='stage').repeat(n).reset_index(drop=True)

lambdas_pitches = np.arange(50, 9, -10)
lambdas_views = np.arange(30, 9, -5)
pitches = pd.Series(
    np.random.poisson(lambdas_pitches, (n, 5)).flatten(order='F'), name='pitches')
views = pd.Series(
    np.random.poisson(lambdas_views, (n, 5)).flatten(order='F'), name='views')

df = pd.concat([item_names, stages, pitches, views],
               axis=1)

df.sample(5)

,item_name,stage,pitches,views
110,Item 10,MQL,32,33
13,Item 13,Prospect,49,30
465,Item 65,Closed Won,7,10
144,Item 44,MQL,43,27
192,Item 92,MQL,34,31


In [3]:
df['view2pitch_ratio'] = df['views']/df['pitches']

In [4]:
# engage_stat['ContactStatus'].unique()

stages = ['Prospect', 'MQL', 'SQL', 'Opportunity', 'Closed Won']

top_pitched_names = np.chararray(
    (10, len(stages)), itemsize=n, unicode=True)
top_pitched_pitches = np.empty([10, len(stages)])
top_pitched_views = np.empty([10, len(stages)])
top_pitched_v2p = np.empty([10, len(stages)])


for stage in stages:
    i = stages.index(stage)
    top_pitched = df[df['stage'] == stage].sort_values(
        by='pitches', ascending=False)[:10]
    top_pitched_names[:, i] = top_pitched['item_name'].values
    top_pitched_pitches[:, i] = top_pitched['pitches'].values
    top_pitched_views[:, i] = top_pitched['views'].values
    top_pitched_v2p[:, i] = top_pitched['view2pitch_ratio'].values

In [8]:
# Create the content heatmap

# Display pitch count, view count, and view-to-pitch ratio on hover
hover = list(range(len(top_pitched_v2p)))
for x in range(len(top_pitched_v2p)):
    hover[x] = ['Pitches: ' + str(int(i)) + '<br>' + 'Views: ' + str(int(j)) + '<br>' + 'V2P: ' + str(round(k, 2))
                for i, j, k in zip(top_pitched_pitches[x], top_pitched_views[x], top_pitched_v2p[x])]

# set colorscale

colorscale = [[0, '#0f7f3f'],  # green
              [0.5, '#ffff0f'],  # yellow
              [1, '#bf0f0f']]  # red

# Make Annotated Heatmap
pt = ff.create_annotated_heatmap(z=top_pitched_v2p,
                                 zmin=0,
                                 zmax=1,
                                 x=stages,
                                 y=[i for i in np.arange(1, 11)],
                                 xgap=1,
                                 ygap=1,
                                 annotation_text=top_pitched_names,
                                 text=hover,
                                 colorscale=colorscale,
                                 font_colors=['black'],
                                 showscale=True,
                                 hoverinfo='text')

pt.layout.title = 'Content Heatmap'

pt.layout.height = 700
pt.layout.width = 1000

pt.layout.xaxis.showgrid = False
pt.layout.yaxis.showgrid = False
pt.layout.xaxis.showline = False
pt.layout.yaxis.showline = False
pt.layout.xaxis.zeroline = False
pt.layout.yaxis.zeroline = False
pt.layout.yaxis.autorange = 'reversed'
pt.layout.yaxis.title = 'Most pitched items'

iplot(pt)